In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display
from scipy.stats import mannwhitneyu, wilcoxon

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.getcwd()
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.bulk_metrics as bulk_metrics

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [2]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [3]:
dataDB = DataFCDatabase(params)

Reading channel label file
Reading channel area file
Reading allen brain map
Reading task structure
Reading session structure
Searching for data files
Found mice ['mou_5', 'mou_6', 'mou_7', 'mou_9']


In [4]:
ds = DataStorage('gallerosalas_bulk_metrics.h5')

In [5]:
mc = MetricCalculator(serial=True, verbose=False)

In [6]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

mice {'mou_9', 'mou_5', 'mou_7', 'mou_6'}
nSessions 4
datatypes ['raw', 'bn_session', 'bn_trial']
nChannel 27


In [7]:
dataDB.calc_shortest_distances()

# Bulk Metrics

In [8]:
intervNames = ["PRE", "TEX", "DEL", "REW", "AVG"]

In [9]:
exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
    {'mousename' : 'mou_6', 'intervName' : 'AVG'},    # Makes no sense to compare: no reward and longer delay
]

## 1. Mean
### 1.1. Mean vs time

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, 'mean', 's', 'time', verbose=False, minTrials=10,
                               trialTypeNames='auto', perfNames=None, exclQueryLst=exclQueryLst)

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'mean', 'time', verbose=False, xFunc=lambda m, l: dataDB.get_times())

### 1.2. Mean vs session

In [10]:
bulk_metrics.metric_mouse_bulk_vs_session(dataDB, mc, ds, "mean", 'session', trialTypeNames='auto',
                                          minTrials=10, verbose=False, intervNames=intervNames,
                                          exclQueryLst=exclQueryLst)

IntProgress(value=0, description='session', max=250)

Skipping existing session_rs_raw_PRE_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_TEX_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_DEL_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_Hit_PRE_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_Hit_TEX_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_Hit_DEL_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_Miss_PRE_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_Miss_TEX_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_Miss_DEL_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_CR_PRE_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_CR_TEX_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_CR_DEL_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_FA_PRE_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_FA_TEX_mou_6_mean_(sessions,)
Skipping existing session_rs_raw_FA_DEL_mou_6_mean_(sessions,)
Skipping existing session_bn_session_PRE_mou_6_mean_(se

In [ ]:
bulk_metrics.scatter_metric_bulk(ds, 'mean', 'session', verbose=False, xFunc=None, haveRegression=False)

### 1.3 Conditions

In [11]:
bulk_metrics.barplot_conditions(ds, 'mean', 'session', verbose=True,
                                trialTypes=['Hit', 'CR'],
                                intervNames=dataDB.get_interval_names())

bn_session
bn_trial
raw


## 2. Variance

**TODO**
* Plot trial variance relative to temporal variance
* Plot mean with variance together

### Pros/Cons of Baseline Normalization
* DFF-Trial
    - Pos: Removes dynamic baseline changing on the order of trials.
    - Pos: Under assumption of signal-free pre-trial interval, baseline removal enhances relative change in significant activity during trial.
    - Neg: In presence of correlation between pre-trial interval and trial signals, this procedure destroys information during trial.

* DFF-Session vs ZScore-Session
    - Both linear transforms
    - Mean is more meaningful for DFF if pre-trial interval is at least somewhat stable
    - Va

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, 'varmean', 's', 'time', verbose=False, minTrials=50,
                               trialTypeNames='auto', exclQueryLst=exclQueryLst)

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'varmean', 'time',
                              ylim=[0,None], verbose=False, xFunc=lambda m, l: dataDB.get_times())

### Session

In [12]:
bulk_metrics.metric_mouse_bulk_vs_session(dataDB, mc, ds, "varmean", 'session',
                                          trialTypeNames=[None, 'Hit', 'CR'],
                                          minTrials=30, skipExisting=True,
                                          verbose=False, intervNames=intervNames,
                                          exclQueryLst=exclQueryLst)

IntProgress(value=0, description='session', max=150)

IntProgress(value=0, description='mou_6', max=30)

IntProgress(value=0, description='mou_6', max=30)

IntProgress(value=0, description='mou_6', max=30)

IntProgress(value=0, description='mou_6', max=30)

...

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

In [13]:
bulk_metrics.scatter_metric_bulk(ds, 'varmean', 'session', verbose=False, xFunc=None, haveRegression=False)

['varmean', 'session', 'mou_5', 'dset453', 'bn_session', 'AVG', 'None', 'CR']
(21,)
(20,)
['varmean', 'session', 'mou_7', 'dset495', 'bn_session', 'AVG', 'None', 'CR']
(25,)
(25,)
['varmean', 'session', 'mou_9', 'dset537', 'bn_session', 'AVG', 'None', 'CR']
(20,)
(18,)
['varmean', 'session', 'mou_5', 'dset465', 'bn_trial', 'AVG', 'None', 'CR']
(21,)
(20,)
['varmean', 'session', 'mou_7', 'dset507', 'bn_trial', 'AVG', 'None', 'CR']
(25,)
(25,)
['varmean', 'session', 'mou_9', 'dset549', 'bn_trial', 'AVG', 'None', 'CR']
(20,)
(18,)
['varmean', 'session', 'mou_5', 'dset438', 'raw', 'AVG', 'rs', 'CR']
(21,)
(20,)
['varmean', 'session', 'mou_7', 'dset480', 'raw', 'AVG', 'rs', 'CR']
(25,)
(25,)
['varmean', 'session', 'mou_9', 'dset522', 'raw', 'AVG', 'rs', 'CR']
(20,)
(18,)
['varmean', 'session', 'mou_5', 'dset451', 'bn_session', 'DEL', 'None', 'CR']
(21,)
(20,)
['varmean', 'session', 'mou_6', 'dset417', 'bn_session', 'DEL', 'None', 'CR']
(30,)
(29,)
['varmean', 'session', 'mou_7', 'dset493', 

['varmean', 'session', 'mou_5', 'dset443', 'bn_session', 'AVG', 'None', 'None']
(21,)
(21,)
['varmean', 'session', 'mou_7', 'dset485', 'bn_session', 'AVG', 'None', 'None']
(25,)
(25,)
['varmean', 'session', 'mou_9', 'dset527', 'bn_session', 'AVG', 'None', 'None']
(20,)
(20,)
['varmean', 'session', 'mou_5', 'dset457', 'bn_trial', 'AVG', 'None', 'None']
(21,)
(21,)
['varmean', 'session', 'mou_7', 'dset499', 'bn_trial', 'AVG', 'None', 'None']
(25,)
(25,)
['varmean', 'session', 'mou_9', 'dset541', 'bn_trial', 'AVG', 'None', 'None']
(20,)
(20,)
['varmean', 'session', 'mou_5', 'dset428', 'raw', 'AVG', 'rs', 'None']
(21,)
(21,)
['varmean', 'session', 'mou_7', 'dset470', 'raw', 'AVG', 'rs', 'None']
(25,)
(25,)
['varmean', 'session', 'mou_9', 'dset512', 'raw', 'AVG', 'rs', 'None']
(20,)
(20,)
['varmean', 'session', 'mou_5', 'dset441', 'bn_session', 'DEL', 'None', 'None']
(21,)
(21,)
['varmean', 'session', 'mou_6', 'dset411', 'bn_session', 'DEL', 'None', 'None']
(30,)
(30,)
['varmean', 'session'

### Conditions

In [18]:
bulk_metrics.barplot_conditions(ds, 'varmean', 'session',
                                verbose=True,
                                trialTypes=['Hit', 'CR'],
                                intervNames=['PRE', 'DEL', 'TEX', 'REW'])

bn_session
bn_trial
raw


### Test variance across channels for interesting interval

* Average signal over texture presentation interval
* Compute variance over trials for each channel
* Compare channels

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, 'varmean', 'p', 'channel',
                               cropTime=('TEX', (3, 3.5)), verbose=False)

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'varmean', 'channel', yscale='log', verbose=False, # ylim=[0.005,2],
                              dropCols=['cropTime'])

# 3. Effective Rank

### ByTime

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "rank_effective", 's', 'time', verbose=False,
                               minTrials=50, trialTypeNames='auto', exclQueryLst=exclQueryLst)

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'rank_effective', 'time', ylim=[1, None], verbose=False,
                              xFunc=lambda m, l: dataDB.get_times()) # ylim=[1,48]

### BySession

In [16]:
bulk_metrics.metric_mouse_bulk_vs_session(dataDB, mc, ds, "rank_effective", 'session',
                                          minTrials=30,
                                          trialTypeNames=[None, 'Hit', 'CR'],
                                          verbose=False,
                                          intervNames=intervNames,
                                          exclQueryLst=exclQueryLst, skipExisting=True)

IntProgress(value=0, description='session', max=150)

IntProgress(value=0, description='mou_6', max=30)

IntProgress(value=0, description='mou_6', max=30)

IntProgress(value=0, description='mou_6', max=30)

IntProgress(value=0, description='mou_6', max=30)

...

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

IntProgress(value=0, description='mou_9', max=20)

In [ ]:
bulk_metrics.scatter_metric_bulk(ds, 'rank_effective', 'session', ylim=[1, None], verbose=False,
                                 xFunc=None, haveRegression=False)

### Conditions

In [17]:
bulk_metrics.barplot_conditions(ds, 'rank_effective', 'session',
                                verbose=True, trialTypes=['Hit', 'CR'],
                                intervNames=dataDB.get_interval_names())

bn_session
bn_trial
raw


## Hierarchical

**TODO**:
* Add composite selectors for metric helper
   - trialType = iGO & iNOGO
   - cropTime = TEX & REW

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "rank_effective", '', 'flat', verbose=False,
                               trialTypeNames='auto',
                               perfNames='auto',
                               intervNames=intervNames, exclQueryLst=exclQueryLst)

In [ ]:
df = ds.list_dsets_pd()
df = df[df['name'] == 'flat']

In [ ]:
data = [np.round(ds.get_data(dset), 2) for dset in df['dset']]
df['rez'] = data

In [ ]:
df.drop(['datetime', 'zscoreDim', 'dset', 'metric', 'name', 'shape', 'target_dim'], axis=1, inplace=True)

In [ ]:
from mesostat.utils.pandas_helper import pd_category_to_column

In [ ]:
index = list(set(df.columns) - {'mousename', 'rez'})
dfMouse = df.pivot(index=index, columns='mousename', values='rez').reset_index()

In [ ]:
dfMouse[dfMouse['datatype'] == 'bn_trial']

# Total Correlation

In [ ]:
# bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "avg_entropy", 'sp', 'time-channel', verbose=False)
# bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "avg_entropy", 's', 'time', verbose=False)

bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "avg_TC", 's', 'time', verbose=False)

In [ ]:
bulk_metrics.plot_metric_bulk(dataDB, ds, 'avg_TC', 'time', verbose=True) # ylim=[1,48]

In [ ]:
bulk_metrics.plot_TC(dataDB, ds, ylim=None, yscale=None, verbose=True)

### Cleaning up

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ds.list_dsets_pd().sort_values(by='datetime'))

In [ ]:
ds.delete_by_query(queryDict={"metric" : "rank_effective"}, timestr="2020-11-20 18:00:00")

# Time-Continuous

In [ ]:
intervNames = ["PRE", "TEX", "DEL", "REW", "AVG"]

exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
    {'mousename' : 'mou_6', 'intervName' : 'AVG'},    # Makes no sense to compare: no reward and longer delay
]

dataDB.